In [1]:
print(2+3)

5


In [2]:
import os
print(os.getcwd())

c:\Users\GenaiblrpioUsr2\Desktop\Team24


In [24]:
import os

# file = r"C:\Users\GenaiblrpioUsr2\Desktop\Team24\database\data\inventory.xlsx"
file = r"C:\Users\GenaiblrpioUsr2\Desktop\Team24\database\data\pdfs"

if os.path.exists(file):
    print("File found ✅")
else:
    print("File not found ❌")

File found ✅


In [14]:
import pandas as pd

def load_inventory():
    file = r"C:\Users\GenaiblrpioUsr2\Desktop\Team24\database\data\inventory.csv"
    df = pd.read_csv(file)
    
    # normalize column names to lowercase
    df.columns = [c.lower() for c in df.columns]
    
    inv = {} 
    for _, r in df.iterrows():
        inv[r["item_id"].upper()] = {
            "name": r["product_name"],
            "stock": int(r["quantity"]),
            "threshold": int(r["threshold"]),
            "category": r["category"],
            "unit_price": r["unit_price"],
            "supplier": r["supplier"],
            "warehouse" : r["warehouse"]
        }
    return inv

# Test the function
inventory = load_inventory()
for sku, details in inventory.items():
    print(f"{sku}: {details}")


SKU101: {'name': 'Laptop', 'stock': 25, 'threshold': 10, 'category': 'Electronics', 'unit_price': 800, 'supplier': 'TechWorld', 'warehouse': 'Warehouse A'}
SKU102: {'name': 'Chair', 'stock': 120, 'threshold': 50, 'category': 'Furniture', 'unit_price': 45, 'supplier': 'FurniCo', 'warehouse': 'Warehouse B'}
SKU103: {'name': 'Printer Ink', 'stock': 8, 'threshold': 15, 'category': 'Office Supply', 'unit_price': 20, 'supplier': 'InkHouse', 'warehouse': 'Warehouse A'}
SKU104: {'name': 'Desk Lamp', 'stock': 60, 'threshold': 30, 'category': 'Furniture', 'unit_price': 25, 'supplier': 'BrightCo', 'warehouse': 'Warehouse C'}
SKU105: {'name': 'USB Cable', 'stock': 12, 'threshold': 20, 'category': 'Electronics', 'unit_price': 5, 'supplier': 'CableMart', 'warehouse': 'Warehouse A'}


In [9]:
inventory = load_inventory()   # ✅ Call the function
for k, v in inventory.items():
    print(f"{k}: {v}")

LAPTOP: {'name': 'Electronics', 'stock': 10, 'threshold': 800, 'category': 25, 'unit_price': 'TechWorld', 'supplier': 'Warehouse A'}
CHAIR: {'name': 'Furniture', 'stock': 50, 'threshold': 45, 'category': 120, 'unit_price': 'FurniCo', 'supplier': 'Warehouse B'}
PRINTER INK: {'name': 'Office Supply', 'stock': 15, 'threshold': 20, 'category': 8, 'unit_price': 'InkHouse', 'supplier': 'Warehouse A'}
DESK LAMP: {'name': 'Furniture', 'stock': 30, 'threshold': 25, 'category': 60, 'unit_price': 'BrightCo', 'supplier': 'Warehouse C'}
USB CABLE: {'name': 'Electronics', 'stock': 20, 'threshold': 5, 'category': 12, 'unit_price': 'CableMart', 'supplier': 'Warehouse A'}


In [2]:
from langchain_openai import ChatOpenAI
import httpx

# Create an HTTP client that skips SSL verification (only for hackathon/test environments)
client = httpx.Client(verify=False)
llm = ChatOpenAI(
 base_url="https://genailab.tcs.in",
 model="azure/genailab-maas-gpt-4o",
 api_key="sk-u6zTQaiDKlhHn4-k_hhihw",
 http_client=client
)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(
 base_url="https://genailab.tcs.in",
 model="azure/genailab-maas-text-embedding-3-large",
 api_key="sk-u6zTQaiDKlhHn4-k_hhihw",
 http_client=client)

import requests
for method in ("get","post","put","delete","head","options","patch"):
    original = getattr(requests,method)

    def insecure_request(*args, _original = original, **kwargs):
        kwargs["verify"] = False
        return _original(*args,**kwargs)
    
    setattr(requests,method,insecure_request)

In [18]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableMap
from database.loader import load_pdfs

pdf_folder =  r"C:\Users\GenaiblrpioUsr2\Desktop\Team24\database\data\pdfs"
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]

# Load each PDF and store chunks
all_docs1 = []

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()  # List[Document]
    
    # Optionally, add source info to each document
    for doc in docs:
        doc.metadata["source"] = os.path.basename(pdf_file)
        all_docs1.append(doc)
        
full_text1 = "\n".join([doc.page_content for doc in all_docs1])

txt_folder = r"C:\Users\GenaiblrpioUsr2\Desktop\Team24\database\data\txts"
txt_files = [os.path.join(txt_folder, f) for f in os.listdir(txt_folder) if f.lower().endswith(".txt")]

all_docs = []

for txt_file in txt_files:
    with open(txt_file, "r", encoding="utf-8") as f:
        content = f.read()

            # Create a simple document-like dict (similar to PDF loader output)
        doc = {
            "page_content": content,
            "metadata": {"source": os.path.basename(txt_file)}
        }
    all_docs.append(doc)

 # Combine all text into one big string
full_text = "\n".join([doc["page_content"] for doc in all_docs])

combined = full_text + full_text1

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that chunks documents for semantic search."),
    ("human", """Split the following document into semantically meaningful sections. 
Follow these rules:
1. Do not omit any information — every detail must appear in some chunk.
2. Keep related items (like lists of dates, events, or tables) together in the same chunk.
3. Each chunk should be self-contained and understandable without needing other chunks.
4. Limit chunk size to about 5000–6000 words. If longer or Lesser, split carefully without breaking sentences or lists.
5. Return the chunks as a numbered list, with each chunk clearly separated.

Document:
{document}""")
])

formatted_prompt = prompt.format_messages(document=combined)

response = llm.invoke(formatted_prompt)
chunks = response.content.split("\n\n")  # crude split; refine if needed

chunked_docs = [Document(page_content=chunk.strip()) for chunk in chunks if chunk.strip()]


vector_store = Chroma.from_documents(
    chunked_docs,
    embedding_model,
    persist_directory="chroma_db"
)
vector_store.persist()


retriever = vector_store.as_retriever(search_kwargs={"k": 3})
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on retrieved documents."),
    ("human", """Using the following context, provide a complete answer. 
    Do not omit any relevant details. 
    If multiple chunks contain related information, merge them into a single coherent answer. Don't miss any kind of data.

    Context:
    {context}

    Question: {question}""")
])

# Chain: retrieve → format prompt → invoke LLM
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [25]:
query = "Summary of the content"
response = rag_chain.invoke(query)
print(response.content)

The content indicates that the document has been organized into semantically meaningful sections to provide a clear and logical breakdown. These sections are designed to convey the main themes and insights from the document effectively. This structured approach ensures that the key ideas and information are presented in a coherent and digestible way for the reader.


In [37]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableMap
from database.loader import load_pdfs

pdf_folder =  r"C:\Users\GenaiblrpioUsr2\Desktop\Team24\database\data\pdfs"
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]

# Load each PDF and store chunks
all_docs = []

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()  # List[Document]
    
    # Optionally, add source info to each document
    for doc in docs:
        doc.metadata["source"] = os.path.basename(pdf_file)
        all_docs.append(doc)
        
full_text = "\n".join([doc.page_content for doc in all_docs])

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that chunks documents for semantic search."),
    ("human", "Split the following document into semantically meaningful sections. Return each chunk as a numbered list.\n\n{document}")
])

formatted_prompt = prompt.format_messages(document=full_text)

response = llm.invoke(formatted_prompt)
chunks = response.content.split("\n\n")  # crude split; refine if needed

chunked_docs = [Document(page_content=chunk.strip()) for chunk in chunks if chunk.strip()]


vector_store = Chroma.from_documents(
    chunked_docs,
    embedding_model,
    persist_directory="chroma_db"
)
vector_store.persist()


retriever = vector_store.as_retriever(search_kwargs={"k": 3})
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a market insights retrieval agent."
    "Retrieve relevant information to forecast 12-month sales for Laptop."   
    "Focus on:"
      "1. FMCG sector growth (volume/value growth, GST impacts)"
      "2. Rural vs urban market performance and trends"
      "3. Category-specific insights (staples, HPC, food, etc.)"
      "4. Longitudinal trends over quarters"
      "5. Factors affecting growth (promotions, smaller packs, macro disruptions)"
    "Return only the most relevant chunks with:"
     "- source_id"
      "- chunk_text"
      "- chunk_date if available"
      "- relevance score"),
    ("human", "Answer the question using the following context:\n\n{context}\n\nQuestion: {question}")
])

# Chain: retrieve → format prompt → invoke LLM
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [38]:
query = "Forecast the next 12 months of sales for Laptops"
response = rag_chain.invoke(query)
print(response.content)

Based on the provided context, here are the most relevant insights:

1. **Rural vs Urban Market Performance and Trends**:
   - Rural markets have consistently outpaced urban markets, with higher growth rates observed in previous quarters (e.g., March and September Quarters 2025 with 8.3% and 8.4% growth, respectively). This holds potential for continued demand in rural markets for laptops, particularly if they are positioned as essential productivity tools.

2. **Impact of GST Changes**:
   - GST changes are noted to have slowed FMCG volume growth in the September quarter. For laptops, this could imply a temporary disruption in consumer purchasing behavior until the adjustments stabilize over the next two quarters. However, this impact may ease as inflation shows signs of improving.

3. **Category-Specific and Longitudinal Trends**:
   - While demand for staples remains steady, impulse categories and habit-forming products are seeing a decline, indicating a shift in consumer prioritiza